## References

see README.md

In [2]:
import os
import itertools


import datetime

import editdistance
import numpy as np

import pylab
import matplotlib.pyplot as plt
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks

Using TensorFlow backend.


In [3]:


np.random.seed(55)

In [4]:
import generator_text_image as fake



In [52]:
# verify what we have with existing generator
def lets_understand():
    
    img_w = 128
    img_h = 64
    pool_size = 2
    words_per_epoch = 16000
    val_split = 0.2
    val_words = int(words_per_epoch * (val_split))
    minibatch_size = 32
    
    print("INPUT to generator:")
    print()
    print(K.image_data_format())
    if K.image_data_format() == 'channels_first':
        input_shape = (1, img_w, img_h)
    else:
        input_shape = (img_w, img_h, 1)

    fdir = os.path.dirname(get_file('wordlists.tgz',
                                    origin='http://www.mythic-ai.com/datasets/wordlists.tgz', untar=True))

    print("img_w", img_w)
    print("img_h", img_h)
    print("pool_size", pool_size)
    print("words_per_epoch", words_per_epoch)
    print("val_words", val_words)
    img_gen = fake.TextImageGenerator(monogram_file=os.path.join(fdir, 'wordlist_mono_clean.txt'),
                                 bigram_file=os.path.join(fdir, 'wordlist_bi_clean.txt'),
                                 minibatch_size=minibatch_size,
                                 img_w=img_w,
                                 img_h=img_h,
                                 downsample_factor=(pool_size ** 2),
                                 val_split=words_per_epoch - val_words
                                 )
    
    print()
    print("OUTPUT to generator:")
    print()
    print("img_gen.get_output_size()", img_gen.get_output_size())
    print("img_gen.absolute_max_string_len", img_gen.absolute_max_string_len)
    img_gen.on_train_begin()
    
    temp = img_gen.next_train() # output is dummy data according to comments
    
    for i in temp:
        temp_input, temp_output_is_dummy = i
        
        print()
        print('this corresponds to cnn_rnn_model Input(name=the_input')
        print("the_input is a set of images ", temp_input['the_input'].shape)
        
        print()
        print("source_str for visualization", temp_input['source_str'][:8])
        
        print()
        print("the next are inputs to the model for ctc calculation")
        print()
        print("this corresponds to cnn_rnn_model Input(name='the_labels'")
        print("the_labels are char index from a, ex:t=19")
        print(temp_input['the_labels'][:2])
        
        print()
        print("this corresponds to cnn_rnn_model Input(name='input_length'")
        print('input_length of ctc is 2 short, see ctc_drop_first_2')
        print(temp_input['input_length'][:2])
        
        print()
        print("this corresponds to cnn_rnn_model Input(name='label_length'")
        print('label_length is number of chars in word')
        print(temp_input['label_length'][:8])
        
        break
    


In [51]:
lets_understand()

INPUT to generator:

channels_last
img_w 128
img_h 64
pool_size 2
words_per_epoch 16000
val_words 3200

OUTPUT to generator:

img_gen.get_output_size() 28
img_gen.absolute_max_string_len 16

this corresponds to cnn_rnn_model Input(name=the_input
the_input is a set of images  (32, 128, 64, 1)

source_str for visualization ['the', 'rmp', 'of', 'arpt', 'and', 'axon', 'to', 'pdo']

the next are inputs to the model for ctc calculation
this corresponds to cnn_rnn_model Input(name='the_labels'
the_labels are char index from a, ex:t=19
[[19.  7.  4. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [17. 12. 15. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]]

this corresponds to cnn_rnn_model Input(name='input_length'
input_length of ctc is 2 short, see ctc_drop_first_2
[[30.]
 [30.]]

this corresponds to cnn_rnn_model Input(name='label_length'
label_length is number of chars in word
[[3.]
 [3.]
 [2.]
 [4.]
 [3.]
 [4.]
 [2.]
 [3.]]
